In [ ]:
import sys
sys.path.insert(0, "..")

# Data access

All data is presented as CSV files, most of it compressed into `tar.gz` files. For convenient access in Python you can use: 

In [ ]:
from src.data import Data, Metrics

## Direct table access

Each source (identified by `source_id`) is published as a single CSV file for each week. The CSVs contain the **free dates** that have been reported by the website **at each snapshot time** (typically every 15 minutes) for **each sub-location** ((identified by `location_id`) of the source. Those sub-locations typically are separate offices but can also represent the same office through different schedules or locations in different cities.

`source_id` and `location_id` together form the unique ID of one calendar, sampled at the snapshot `date`. 

Individual tables can be accessed with the `Data.get_table` class method:

In [ ]:
# get table for specific calendar week and source_id
columns, rows = Data.get_table((2021, 30), "leipzig")
print(columns[:5])
print(rows[0][:5])

Or with the `Data.get_dataframe` class method which returns a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html):

In [ ]:
Data.get_dataframe((2021, 30), "leipzig")

For convenience, both methods support adding the names from the [metadata.json](compressed/metadata.json) file:

In [ ]:
Data.get_dataframe((2021, 30), "leipzig", with_meta=True)

Do not worry about all the zeros in there. Look at ["Plotting the raw data"](#plotting-the-raw-data) to see the free dates in between them. 

Dataframes are very useful to analyze the data. For example, we can count the sum of all free dates available for each sub-location with these three lines:

In [ ]:
df = Data.get_dataframe((2021, 30), "leipzig", with_meta=True)
# sum over each location and accross all available dates 
loc_sum = df.groupby("location_name").sum().sum(axis=1)
# sort and plot
loc_sum.sort_values().plot.barh(figsize=(10, 6))

The `date`, `source_id` and `location_id` (as well as `source_name` and `location_name`) columns form a [pandas.MultiIndex](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.html#pandas.MultiIndex) (See [multiindexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#)). The individual values can be accessed like this:

In [ ]:
df.index.get_level_values("date")

### Note on filters

All string filters (for `source_id`, `location_id` or [metric name](#precalculated-metrics-access)) support:

- [wildcard matching](https://en.wikipedia.org/wiki/Wildcard_character#File_and_directory_patterns). e.g. `"bonn*"` would match `"bonn"` and `"bonnbau"`
- lists of wildcards. e.g. `["bonn*", "dresden*"]`
- functions. e.g. `lambda n: n.startswith("bonn")`

## Iterate through the data

To access all tables in the published dataset you can use the `Data.iter_*` methods. The `Data` class has to be instantiated and will accept *filters* in the constructor:

In [ ]:
data = Data(source_id=["jena", "bonn*"], iso_week_lt=(2021, 30))

for week, source_id, fileio in data.iter_files():
    print(f"{week} {source_id:10s} {len(fileio.read())} bytes")

The `fileio` parameter is a readable binary stream into each file inside each `tar.gz` archive.

The `Data.iter_tables` generator will yield the table data instead:

In [ ]:
for week, source_id, columns, rows in data.iter_tables():
    print(f"{week} {source_id:10s} {len(rows)} rows x {len(columns)} columns")

And `iter_dataframes` will yield a `pandas.DataFrame` for each table:

In [ ]:
for week, source_id, df in data.iter_dataframes():
    print(f"{week} {source_id:10s} shape={df.shape}")

Note that the dataframes have three columns less than the raw tables because the `date`, `source_id` and `location_id` columns are moved to the pandas multiindex.

The yielded tables or dataframes are always sorted first by week and secondary by source_id. From these tables, it's possible to calculate all kinds of metrics, although it takes some time as the uncompressed tables are gigabytes in size altogether. 

For the impatient there are some precalcuated metrics contained in this repository.

# Precalculated metrics access

The available metrics are `free_dates`, `appointments` and `cancellations`. The table rows are the snapshot dates (truncated to exact 15 minutes steps) and the columns contain each metric for each source and sub-location. The column names are concatenated strings of `source_id`/`location_id`/`metric_name`.

The above example of free dates per sub-location can also be calculated from the metrics using:

In [ ]:
df = Metrics.dataframe("free_dates", "leipzig", iso_week=(2021, 30))

which gives us the filtered metrics:

In [ ]:
df

which can likewise be summed and plotted:

In [ ]:
df.sum().sort_values().plot.barh(figsize=(10, 6))

### Appointments and cancellations

The number of appointments that have been made between two website snapshots, as well as the number of cancelled appointments are estimated from the raw data. 

For example, we can plot these metrics for all locations of a source, summed for each day:

In [ ]:
df = Metrics.dataframe(
    ["appointments", "cancellations"], "blankenburg",
    iso_week_gte=(2021, 30), iso_week_lte=(2021, 37),  # put a time range to the returned data
)
df.resample("1d").sum().plot(figsize=(16, 4))

### Metrics with timespans

All of the metrics are also available according to their distance to the snapshot time. For example, `appointments_0d` holds the number of appointments made at the same day as the snapshot, or `free_dates_1h` holds the number of free dates that are 1 hour apart from the snapshot date. Snapshot and possible appointment dates are quantized
to full hours, days or calendar weeks before calculating the distance.

Using wildcard matching, we can retrieve all timespans for a certain metric: 

In [ ]:
df = Metrics.dataframe(
    ["appointments_*w"], "blankenburg", "85150", 
    iso_week_gte=(2021, 30), iso_week_lte=(2021, 37),
)
df.resample("1d").sum().plot(figsize=(16, 4))

We can see that appointments made in the same week (`appointments_0w`) are quite rare, and appointments within the next week (`appointments_1w`) are getting rarer while appointments made in the week after next week (`appointments_2w`) are growing.

If we compare with the number of free dates available within each weekly timespan, it kind of makes sense and probably is a result of people coming home from holidays and getting busy again:

In [ ]:
df = Metrics.dataframe(
    ["free_dates_*w"], "blankenburg", "85150",
    iso_week_gte=(2021, 30), iso_week_lte=(2021, 37),
)
df.resample("1d").sum().plot(figsize=(16, 4))

## Plotting the raw data

Still, something seems not right there. Best to have a look at the actual free dates. Because it's such an enormous amount of data, a table does not work well. But we can plot something like a heatmap. 

First find the calendar week of interest. The plot has labeled the 23rd of August in the area of interest. The calendar week is:

In [ ]:
import datetime
datetime.date(2021, 8, 23).isocalendar()

Week 34. Next get a dataframe resampled to 1 hour steps, both for snapshot dates as for free dates, to make it fit better into a plot:

In [ ]:
import pandas as pd
import numpy as np

df = (
    # examine the week before and after as well
    pd.concat([
        Data.get_dataframe((2021, 33), "blankenburg", "85150"),
        Data.get_dataframe((2021, 34), "blankenburg", "85150"),
        Data.get_dataframe((2021, 35), "blankenburg", "85150"),
    ])
    # just keep the "date" index
    .droplevel(["source_id", "location_id"])
    # resample snapshot date to 1 hour steps
    .resample("1h").sum()
    # resample possible appointment date to 1 hour steps
    .resample("1h", axis=1).sum()
    # replace zeros with NaNs which are not plotted
    # and make the graphic much more readable
    .replace(0, np.nan)
)

I'll use the [plotly](https://plotly.com/python/) library to produce the heatmap because it's less complicted compared to matplotlib. Also, in the jupyter notebook, it's interactive and allows zooming and tooltips per value.

In [ ]:
import plotly.express as px

fig = px.imshow(
    df, aspect="auto", width=1000, height=700, 
    labels={"x": "appointment date", "y": "snapshot date"},
)

In the notebook, one could use `fig.show()` to display the interactive plot. However, this does not work in the markdown conversion so we convert the javascript plot to an image (using [kaleido](https://github.com/plotly/Kaleido) in the back).

In [ ]:
from IPython.display import Image
Image(fig.to_image())

Free appointment dates are those vertical lines that come from the top and vanish when either the date has been taken by someone or it's getting close to the actual snapshot time, which means that the date passed unused.

Obviously, there is problem here. The free dates in September are not visible in August. Good that we had a look! A [fix](https://github.com/defgsus/office-schedule-scraper/commit/d66d9b815be316857027d501d1131cec7b53f475) is applied, but all **etermin**-based scrapers have produced incomplete snapshots up until now.